# 🛳 Titanic Data Preprocessing & Logistic Regression Model
This notebook covers:
- Loading the Titanic dataset
- Exploring dataset structure
- Handling missing values
- Encoding categorical variables
- Removing duplicates like `embarked` vs `embark_town`
- Handling outliers
- Scaling numerical features
- Feature engineering
- Training a Logistic Regression model
- Evaluating performance

All previous errors (like correlation errors due to non-numeric columns) are handled properly.

## 1️⃣ Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

%matplotlib inline
print('Libraries imported successfully!')

## 2️⃣ Load the Titanic Dataset

In [ ]:
data = sns.load_dataset('titanic')
data.head()

## 3️⃣ Explore the Dataset

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

## 4️⃣ Handle Missing Values
We fill missing values in:
- `age` → median
- `embarked` → mode
- Drop `deck` because it has too many missing values

In [ ]:
data['age'].fillna(data['age'].median(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data.drop(columns=['deck'], inplace=True)
data.isnull().sum()

## ❓ Why Not Keep Both `embarked` and `embark_town`?
Both represent where the passenger boarded. To avoid duplication and correlation issues, we drop `embark_town`.

In [ ]:
data.drop(columns=['embark_town'], inplace=True)
data.isnull().sum()

## 5️⃣ Encode Categorical Variables
We convert:
- `sex` → 0/1
- `adult_male`, `alone` → int
- `embarked` → one-hot encoded

In [ ]:
data['sex'] = data['sex'].map({'male': 0, 'female': 1})
data['adult_male'] = data['adult_male'].astype(int)
data['alone'] = data['alone'].astype(int)

data = pd.get_dummies(data, columns=['embarked'], drop_first=True)
data.head()

## 6️⃣ Remove Other Non-Numeric or Duplicate Columns
`alive`, `class`, and `who` are duplicates or categorical labels not needed for prediction.

In [ ]:
data.drop(columns=['alive','class','who'], inplace=True)
data.head()

## 7️⃣ Detect & Handle Outliers in Fare

In [ ]:
sns.boxplot(data['fare'])
plt.title('Fare Outliers')
plt.show()

In [ ]:
Q1 = data['fare'].quantile(0.25)
Q3 = data['fare'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

data = data[(data['fare'] >= lower) & (data['fare'] <= upper)]
data.shape

## 8️⃣ Feature Scaling

In [ ]:
scaler = StandardScaler()
data[['age','fare']] = scaler.fit_transform(data[['age','fare']])
data.head()

## 9️⃣ Correlation Heatmap (Fixed Version)
We use only numeric columns to avoid errors like earlier.

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(data.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

## 🔟 Feature Engineering

In [ ]:
data['family_size'] = data['sibsp'] + data['parch'] + 1
data['is_alone'] = np.where(data['family_size']==1,1,0)
data[['family_size','is_alone']].head()

## 1️⃣1️⃣ Split Data into Features and Target

In [ ]:
X = data.drop('survived', axis=1)
y = data['survived']

## 1️⃣2️⃣ Train–Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape

## 1️⃣3️⃣ Train Logistic Regression Model

In [ ]:
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)


## 1️⃣4️⃣ Predictions

In [ ]:
y_pred = model.predict(X_test)
y_pred[:10]

## 1️⃣5️⃣ Model Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))